<a href="https://colab.research.google.com/github/rnomadic/Databricks_ML/blob/main/MLFlow-Model-Registry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
*******************    MLFlow Model Registry    *********************

The MLflow Model Registry component is a centralized model store, set of APIs, 
and UI, to collaboratively manage the full lifecycle of an MLflow Model. 
It provides 
1> model lineage (which MLflow Experiment and Run produced the model), 
2> model versioning, 
3> stage transitions (e.g. from staging to production), 
4> annotations (e.g. with comments, tags), and 
5> deployment management (e.g. which production jobs have requested a specific model version).


Central Repository: Register MLflow models with the MLflow Model Registry. 
A registered model has a unique name, version, stage, and other metadata.

Model Versioning: Automatically keep track of versions for registered models when updated.

Model Stage: Assigned preset or custom stages to each model version, 
like “Staging” and “Production” to represent the lifecycle of a model.

Model Stage Transitions: Record new registration events or changes as activities 
that automatically log users, changes, and additional metadata such as comments.

CI/CD Workflow Integration: Record stage transitions, request, review 
and approve changes as part of CI/CD pipelines for better control and governance.
"""

